In [ ]:
# Importation de la bibliothèque pandas
import pandas as pd

# Lecture du fichier CSV et stockage des données dans un DataFrame
df = pd.read_csv('../dataset.csv')

# Sélection des colonnes 'description' et 'cat1' du DataFrame
df = df[['description', 'cat1']]

# Suppression des lignes où la colonne 'description' est vide
df = df.dropna(subset=['description'])

# Remplacement des retours à la ligne dans la colonne 'description' par des espaces
df['description'] = df['description'].str.replace('\n', ' ')

# Importation de la bibliothèque pour le rééquilibrage des classes
from imblearn.over_sampling import RandomOverSampler

# Initialisation de l'objet pour le rééquilibrage des classes
oversampler = RandomOverSampler(random_state=0)

# Séparation des données en caractéristiques (X) et cibles (y)
X = df['description']
y = df['cat1']

# Rééquilibrage des classes
X_resampled, y_resampled = oversampler.fit_resample(df, y)

# Création d'un nouveau DataFrame avec les données rééquilibrées
df = pd.DataFrame(X_resampled, columns=['description', 'cat1'])

# Importation des bibliothèques pour la visualisation des données
import matplotlib.pyplot as plt
import seaborn as sns

# Comptage du nombre de descriptions par catégorie
categories_count = df['cat1'].value_counts()

# Liste des catégories
categories = categories_count.index

# Création d'un graphique pour visualiser le nombre de descriptions par catégorie
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
sns.barplot(x=categories_count, y=categories, ax=ax)
for a,p in enumerate(ax.patches):
    ax.annotate(f'{categories_count[a]}', (p.get_width(), a), fontsize=12)
plt.xlabel('Nombre de descriptions', fontsize=16)
plt.ylabel('Catégorie', fontsize=16)

# Séparation des données en ensemble d'entraînement et ensemble de test
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True  ,random_state=0)
print("Train set shape: ", df_train.shape)
print("Test set shape: ", df_test.shape)

# Encodage des catégories en nombres
encoded_dict = {
    "Festival": 0,
    "Exposition": 1,
    "Théatre": 2,
    "Détente": 3,
    "Action": 4,
    "Marché": 5,
    "Environnement": 6,
    "Atelier": 7,
    "Fête": 8,
    "Sport": 9,
    "Balade": 10,
    "Visite": 11,
    "Brocante": 12,
    "Spectacle": 13,
    "Conférence": 14,
    "Danse": 15,
    "Jeu": 16,
    "Famille": 17,
    "Concert": 18,
    "Culture": 19
}
df_train['cat1'] = df_train['cat1'].map(encoded_dict)
df_test['cat1'] = df_test['cat1'].map(encoded_dict)

# Affichage des premières lignes de l'ensemble d'entraînement
df_train.head()

# Importation des bibliothèques pour le traitement du langage naturel et le machine learning
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader

# Initialisation du tokenizer et du modèle
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-multilingual-cased", num_labels=20)

# Tokenisation des descriptions
train_encodings = tokenizer(df_train['description'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(df_test['description'].tolist(), truncation=True, padding=True)

# Conversion des données tokenisées en un dataset PyTorch
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels 

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = TextDataset(train_encodings, df_train['cat1'].tolist())
test_dataset = TextDataset(test_encodings, df_test['cat1'].tolist())

# Chargement du dataset dans un DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

from torch.optim import AdamW
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer

# Initialisation de l'optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Initialisation de l'early stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

# Fonction pour l'entraînement du modèle
def train(model, dataloader, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Entraînement du modèle
train(model, train_loader, optimizer, epochs=1)

# Fonction pour tester le modèle
def test(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {correct/total}')

# Test du modèle
test(model, test_loader)

#fonction pr tester 

def predict_category(text):
    # Tokenisation du texte
    encoding = tokenizer(text, truncation=True, padding=True, return_tensors='pt')

    # Prédiction avec le modèle
    with torch.no_grad():
        outputs = model(**encoding)

    # Récupération de la catégorie prédite
    _, predicted = torch.max(outputs.logits, 1)

    # Conversion de la catégorie prédite en texte
    for category, index in encoded_dict.items():
        if index == predicted.item():
            return category

    return None
text = "C'est un festival de musique."
print(predict_category(text))
